##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [40]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

In [57]:
paradigmFile="MGC-150812-30000-1508120226-paradigmes.csv"
goldFile="MGC-150805-paradigmes.csv"

###Préparation des cases du paradigme

In [43]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [58]:
paradigmes=pd.read_csv(paradigmFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [45]:
GOLD=pd.read_csv(goldFile,sep=";",encoding="utf8")
del GOLD[u"Unnamed: 0"]
goldCases=GOLD.columns.values.tolist()
goldCases.remove(u"lexeme")

In [62]:
listeTest=paradigmes.dropna(thresh=3)["lexeme"].values.tolist()
for i,element in enumerate(listeTest):
    print i, element,

0 abaisser 1 abandonner 2 abattre 3 abdiquer 4 abjurer 5 abolir 6 abonder 7 abonner 8 aborder 9 abouler 10 aboutir 11 aboyer 12 abreuver 13 abriter 14 abrutir 15 abréger 16 absenter 17 absorber 18 absoudre 19 abstenir 20 abstraire 21 abuser 22 abîmer 23 accabler 24 accaparer 25 accentuer 26 accepter 27 acclamer 28 acclimater 29 accoler 30 accommoder 31 accompagner 32 accomplir 33 accorder 34 accoster 35 accoter 36 accoucher 37 accouder 38 accoupler 39 accourir 40 accoutumer 41 accrocher 42 accroupir 43 accroître 44 accréditer 45 accueillir 46 acculer 47 accumuler 48 accuser 49 accéder 50 accélérer 51 acharner 52 acheminer 53 acheter 54 achever 55 acoquiner 56 acquiescer 57 acquitter 58 acquérir 59 actionner 60 activer 61 acérer 62 adapter 63 additionner 64 adhérer 65 adjoindre 66 adjuger 67 adjurer 68 admettre 69 administrer 70 admirer 71 admonester 72 adonner 73 adopter 74 adorer 75 adosser 76 adoucir 77 adresser 78 advenir 79 affabuler 80 affadir 81 affaiblir 82 affairer 83 affaisser

In [53]:
paradigmes[sampleCases].stack().value_counts(dropna=True).sum()

14980

In [54]:
paradigmes[sampleCases].dropna(thresh=2).stack().value_counts(dropna=True).sum()

14126

In [56]:
paradigmes.dropna(thresh=3)

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,NaN,NaN,NaN,NaN,abEsa,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,abandonner,NaN,NaN,NaN,NaN,abâdOnEr,abâdOna,NaN,NaN,NaN,...,abâdOne,abâdOne,NaN,abâdOne,NaN,NaN,NaN,abâdOn,abâdOn,abâdOn
3,abattre,NaN,NaN,NaN,NaN,abatir,abati,NaN,NaN,NaN,...,NaN,abaty,abaty,abaty,NaN,abat,NaN,NaN,NaN,abat
7,abonder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,aborder,NaN,NaN,NaN,NaN,abOrdEr,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,abOrde,NaN,NaN,NaN,NaN,NaN,NaN
10,aboutir,NaN,NaN,NaN,NaN,NaN,abuti,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,aboyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,abriter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,abrite,NaN,NaN,NaN,NaN,NaN,NaN
13,abréger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,absorber,NaN,NaN,NaN,NaN,NaN,absOrba,NaN,NaN,NaN,...,absOrbe,absOrbe,NaN,absOrbe,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
GOLD[goldCases].stack().value_counts(dropna=True).sum()

327258

In [49]:
GOLD[goldCases].dropna(thresh=2).stack().value_counts(dropna=True).sum()

327198

In [50]:
GOLD[goldCases].dropna(thresh=2)

,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,fi2S,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abEsam,abEsE,abEsat,abEsa,abEsEr,abEsa,abEs6rô,abEs6rE,abEs6re,abEs6ra,...,abEse,abEse,abEse,abEse,abEsjô,abEs,abEsje,abEs,abEs,abEs
1,abâdOnam,abâdOnE,abâdOnat,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,abâdOn6ra,...,abâdOne,abâdOne,abâdOne,abâdOne,abâdOnjô,abâdOn,abâdOnje,abâdOn,abâdOn,abâdOn
2,abazurdim,abazurdi,abazurdit,abazurdi,abazurdir,abazurdi,abazurdirô,abazurdirE,abazurdire,abazurdira,...,abazurdi,abazurdi,abazurdi,abazurdi,abazurdisjô,abazurdis,abazurdisje,abazurdis,abazurdis,abazurdis
3,abatim,abati,abatit,abati,abatir,abati,abatrô,abatrE,abatre,abatra,...,abaty,abaty,abaty,abaty,abatjô,abat,abatje,abat,abat,abat
4,absEdam,absEdE,absEdat,absEda,absEdEr,absEda,absEd6rô,absEd6rE,absEd6re,absEd6ra,...,NaN,NaN,NaN,absEde,absEdjô,absEd,absEdje,absEd,absEd,absEd
5,abdikam,abdikE,abdikat,abdika,abdikEr,abdika,abdik6rô,abdik6rE,abdik6re,abdik6ra,...,abdike,abdike,abdike,abdike,abdikjô,abdik,abdikje,abdik,abdik,abdik
6,abEkam,abEkE,abEkat,abEka,abEkEr,abEka,abEk6rô,abEk6rE,abEk6re,abEk6ra,...,abEke,abEke,abEke,abEke,abEkjô,abEk,abEkje,abEk,abEk,abEk
7,abEram,abErE,abErat,abEra,abErEr,abEra,abEr6rô,abEr6rE,abEr6re,abEr6ra,...,NaN,NaN,NaN,abEre,abErjô,abEr,abErje,abEr,abEr,abEr
8,abOram,abOrE,abOrat,abOra,abOrEr,abOra,abOr6rô,abOr6rE,abOr6re,abOr6ra,...,abOre,abOre,abOre,abOre,abOrjô,abOr,abOrje,abOr,abOr,abOr
9,abZyram,abZyrE,abZyrat,abZyra,abZyrEr,abZyra,abZyr6rô,abZyr6rE,abZyr6re,abZyr6ra,...,abZyre,abZyre,abZyre,abZyre,abZyrjô,abZyr,abZyrje,abZyr,abZyr,abZyr
